# Todo el proceso con el algoritmo con ceros

En este notebook se va a realizar todo el proceso hecho hasta ahora pero usando otra versión del algoritmo. 

Esta nueva versión consiste en rellenar las dimensiones con 0 hasta que el tamaño de la dimensión sea multiplo del tamaño de la partición. 

De esta forma se simplifica mucho el código con una consequente bajada del tiempo de ejecución.

## Paquetes necesarios

In [1]:
import transformData as td
import matplotlib.pyplot as plt
import numpy as np
import pycblosc2 as cb2
import time as t

## Definición del conjunto de datos

### Datos sin transformación

In [58]:
SHAPE = [256, 256, 256]

SIZE = np.prod(SHAPE)

src = np.arange(SIZE, dtype=np.int32).reshape(SHAPE)

### Datos transformados

In [59]:
start = t.perf_counter()

TSUBSHAPE = [32, 32, 32]

srct = td.tData(src, TSUBSHAPE, inverse=False)

TSHAPE = srct.shape

srct = srct.flatten()

indexation = td.createIndexation(TSHAPE, TSUBSHAPE)

end = t.perf_counter()

ttran_tra = end - start

## Compresión y descompresión de los datos

In [60]:
KB = 1024
MB = 1024 * KB
GB = 1024 * MB

BLOSC_MAX_FILTERS = 5
BLOSC_BLOCKSIZE = 16 * KB
BLOSC_TYPESIZE = 4
BLOSC_CODE = 5

cparams = cb2.blosc2_create_cparams(compcode=BLOSC_CODE, clevel=5, use_dict=0, typesize=BLOSC_TYPESIZE,
                                        nthreads=4, blocksize= BLOSC_BLOCKSIZE, schunk=None,
                                        filters=[0, 0, 0, 0, 1],
                                        filters_meta=[0, 0, 0, 0, 0])

dparams = cb2.blosc2_create_dparams(nthreads=4, schunk=None)

### Datos sin transformación

#### Compresión

In [86]:
start = t.perf_counter()

BSIZE = SIZE * src.dtype.itemsize

schunk = cb2.blosc2_new_schunk(cparams, dparams)
nchunks = cb2.blosc2_append_buffer(schunk, BSIZE, src)

end = t.perf_counter()

torig_com = end - start

#### Descompresión

In [87]:
start = t.perf_counter()

dest = np.zeros(SIZE, dtype=np.int32).reshape(SHAPE)
cb2.blosc2_decompress_chunk(schunk, 0, dest, BSIZE)

res = dest[:,2,:]

cb2.blosc2_free_schunk(schunk)

end = t.perf_counter()

torig_dec = end - start

### Datos transformados

#### Compresión

In [88]:
start = t.perf_counter()

schunk = cb2.blosc2_new_schunk(cparams, dparams)

AUX_SIZE = np.prod(TSUBSHAPE)
AUX_BSIZE = AUX_SIZE * srct.dtype.itemsize

for i in range(TSHAPE[0]//TSUBSHAPE[0]* TSHAPE[1]//TSUBSHAPE[1] * TSHAPE[2]//TSUBSHAPE[2]):
    aux = srct[i * AUX_SIZE:(i+1) * AUX_SIZE]
    nchunks = cb2.blosc2_append_buffer(schunk, AUX_BSIZE, aux)
end = t.perf_counter()

ttran_com = end - start

#### Descompresión

In [89]:
start = t.perf_counter()

SUBPL = (TSHAPE[0], TSUBSHAPE[1], TSHAPE[2])

ind = td.obtainIndex((0,TSHAPE[0]), (2,3), (0,TSHAPE[2]), indexation, TSHAPE, TSUBSHAPE)

dest = np.zeros(np.prod(SUBPL), dtype=np.int32).reshape(SUBPL)

AUX_SIZE = np.prod(TSUBSHAPE)
AUX_BSIZE = AUX_SIZE * srct.dtype.itemsize

aux = np.zeros(AUX_SIZE, dtype=np.int32).reshape(TSUBSHAPE)

for (i, j, k), n in ind:
    cb2.blosc2_decompress_chunk(schunk, n, aux, AUX_BSIZE)
    dest[i:i+TSUBSHAPE[0], 0:TSUBSHAPE[1], k:k+TSUBSHAPE[2]] = aux
    
res2 = dest[:SHAPE[0], 2%TSUBSHAPE[1], :SHAPE[2]]

end = t.perf_counter()

ttran_dec = end - start

## Resultados

In [90]:
print('La compresión de los datos transformados es {:.3f}x más rápida que la original.'.format(torig_com/(ttran_com + ttran_tra)))

print('La descompresión de los datos transformados es {:.3f}x más rápida que la original.'.format(torig_dec/ttran_dec))


La compresión de los datos transformados es 0.011x más rápida que la original.
La descompresión de los datos transformados es 2.601x más rápida que la original.


In [91]:
np.testing.assert_array_equal(res, res2)

In [92]:
res2

array([[     512,      513,      514, ...,      765,      766,      767],
       [   66048,    66049,    66050, ...,    66301,    66302,    66303],
       [  131584,   131585,   131586, ...,   131837,   131838,   131839],
       ...,
       [16581120, 16581121, 16581122, ..., 16581373, 16581374, 16581375],
       [16646656, 16646657, 16646658, ..., 16646909, 16646910, 16646911],
       [16712192, 16712193, 16712194, ..., 16712445, 16712446, 16712447]],
      dtype=int32)

In [93]:
res

array([[     512,      513,      514, ...,      765,      766,      767],
       [   66048,    66049,    66050, ...,    66301,    66302,    66303],
       [  131584,   131585,   131586, ...,   131837,   131838,   131839],
       ...,
       [16581120, 16581121, 16581122, ..., 16581373, 16581374, 16581375],
       [16646656, 16646657, 16646658, ..., 16646909, 16646910, 16646911],
       [16712192, 16712193, 16712194, ..., 16712445, 16712446, 16712447]],
      dtype=int32)